In [25]:
import pickle
import cv2
import numpy as np
from moviepy.editor import *

from keras.layers import *
from keras.models import load_model

In [26]:
path = r"E:\Projects\Baby\Media\Dataset 2.0\Person with Baby\Thif.mp4"
# path = r"E:\Projects\Baby\Final\Thif3.mp4"


In [27]:
#opening the video
cap = cv2.VideoCapture(path)

#import pre-trained model
saved_model = load_model('LSTM_Model.h5')

#declare image height and width
img_ht, img_width = 64, 64

#actions
classes = ["Walk", "Run"]


In [28]:
#used for the prediction of action
def predict_action(video_file_path):
    #read the video
    video_reader = cv2.VideoCapture(video_file_path)

    frames_list = []

    predicted_class_name = ''

    #for each in a 20 seq of frames
    for frame_counter in range(20):
        success, frame = video_reader.read()

        if not success:
            break 

        resized_frame = cv2.resize(frame, (img_ht, img_width))

        normalized_frame = resized_frame/255

        frames_list.append(normalized_frame)

# Reshape frames for LSTM input (samples, time_steps, features)
    reshaped_frames = np.array(frames_list).reshape(1, 20, img_ht * img_width * 3)

    predicted_labels_probabilities = saved_model.predict(reshaped_frames)[0]
    predict_label = np.argmax(predicted_labels_probabilities)
    predicted_class_name = classes[predict_label]

    # \nConfidence: {predicted_labels_prrobabilities[predict_label]}
    print(f'Action Predicted: {predicted_class_name}')

    return predicted_class_name

    video_reader.release()

In [29]:

hog = cv2.HOGDescriptor()
#Support vector Machine used for people detection
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (640, 480))

    boxes, weights = hog.detectMultiScale(frame, winStride=(8, 8))

    boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in boxes])

    for (xA, yA, xB, yB) in boxes:
        cv2.rectangle(frame, (xA, yA), (xB, yB),
                      (0, 255, 0), 2)
    
    # cap.write(frame.astype('uint8'))
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

    (humans, _) = hog.detectMultiScale(frame,
                                   winStride=(5, 5),
                                   padding=(3, 4),
                                   scale=1.21)

    print('Human detected: ', len(humans))

    if(len(humans)==1):
        print("SUSPICIOUS PERSON DETECTED!!!")
        predict_action(path)
        
        if predict_action(path)=="Run":
            img = frame
            cv2.imshow("img", img)
            cv2.waitKey(1)
            cv2.destroyAllWindows()
            break

Human detected:  0
Human detected:  0
Human detected:  0
Human detected:  0
Human detected:  0
Human detected:  0
Human detected:  0
Human detected:  0
Human detected:  0
Human detected:  0
Human detected:  0
Human detected:  0
Human detected:  0
Human detected:  0
Human detected:  0
Human detected:  0
Human detected:  1
SUSPICIOUS PERSON DETECTED!!!
1/1 [==============================] - 1s 1s/step
Action Predicted: Run
1/1 [==============================] - 0s 56ms/step
Action Predicted: Run


In [30]:
classes = ['With Baby', 'Without Baby']

img = img
cv2.imshow("Input Image", img)

img = cv2.resize(img, (224, 224))

r_img = img.reshape(1, 224, 224, 3)

img = img/255.0

nx, ny, nrgb = img.shape
# print(r_img.shape)
img2 = img.reshape(1,(nx*ny*nrgb))

pickled_model = pickle.load(open('random.pkl', 'rb'))
ans=pickled_model.predict(img2)
predicted_class = ans.argmax(axis=-1)
print(classes[predicted_class])
print(ans)

cv2.waitKey(0)
cv2.destroyAllWindows()

With Baby
[1.]
